<a href="https://colab.research.google.com/github/VarunUmapathy/Emotion_Bot/blob/main/Emotion_model_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update && apt install cuda-11-8

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [59.5 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,107 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu 

In [ ]:
!pip install "tensorflow-text==2.11.*"

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 39.6 MB/s eta 0:00:00
  Attempting uninstall: tf-keras
    Found existing installation: tf_keras 2.17.0
    Uninstalling tf_keras-2.17.0:
      Successfully uninstalled tf_k

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()


'/device:GPU:0'

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
! unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
! pip install contractions
import contractions

unzip:  cannot find or open /usr/share/nltk_data/corpora/wordnet.zip, /usr/share/nltk_data/corpora/wordnet.zip.zip or /usr/share/nltk_data/corpora/wordnet.zip.ZIP.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 11.9 MB/s eta 0:00:00


In [ ]:
import os
import re
import string
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import files


uploaded = files.upload()


Saving dev.tsv to dev.tsv
Saving ekman_labels.csv to ekman_labels.csv
Saving emotion_file.txt to emotion_file.txt
Saving hello.mp3 to hello.mp3
Saving test.py to test.py
Saving test.tsv to test.tsv
Saving text.mp3 to text.mp3
Saving train.tsv to train.tsv


In [ ]:
train_dataset = pd.read_csv("/content/train.tsv", sep='\t', header=None, names=['Text', 'Class', 'ID'])
valid_dataset = pd.read_csv("/content/dev.tsv", sep='\t', header=None, names=['Text', 'Class', 'ID'])
test_dataset = pd.read_csv("/content/test.tsv", sep='\t', header=None, names=['Text', 'Class', 'ID'])

In [ ]:
train_dataset.head()

,Text,Class,ID
0,My favourite food is anything I didn't have to...,27,eebbqej
1,"Now if he does off himself, everyone will thin...",27,ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj
3,To make her feel threatened,14,ed7ypvh
4,Dirty Southern Wankers,3,ed0bdzj


In [ ]:
train_dataset['Class List'] = train_dataset['Class'].apply(lambda x: x.split(','))
train_dataset['Class Length'] = train_dataset['Class List'].apply(lambda x: len(x))

valid_dataset['Class List'] = valid_dataset['Class'].apply(lambda x: x.split(','))
valid_dataset['Class Length'] = valid_dataset['Class List'].apply(lambda x: len(x))

test_dataset['Class List'] = test_dataset['Class'].apply(lambda x: x.split(','))
test_dataset['Class Length'] = test_dataset['Class List'].apply(lambda x: len(x))

In [ ]:
train_dataset["Class Length"].value_counts()

,count
Class Length,
1,36308
2,6541
3,532
4,28
5,1


In [ ]:
ekman_mapping = {
  "anger": [
    "anger",
    "annoyance",
    "disapproval"
  ],
  "disgust": [
    "disgust"
  ],
  "fear": [
    "fear",
    "nervousness"
  ],
  "joy": [
    "joy",
    "amusement",
    "approval",
    "excitement",
    "gratitude",
    "love",
    "optimism",
    "relief",
    "pride",
    "admiration",
    "desire",
    "caring"
  ],
  "sadness": [
    "sadness",
    "disappointment",
    "embarrassment",
    "grief",
    "remorse"
  ],
  "surprise": [
    "surprise",
    "realization",
    "confusion",
    "curiosity"
  ]
}

In [ ]:
ekman_mapping

{'anger': ['anger', 'annoyance', 'disapproval'],
 'disgust': ['disgust'],
 'fear': ['fear', 'nervousness'],
 'joy': ['joy',
  'amusement',
  'approval',
  'excitement',
  'gratitude',
  'love',
  'optimism',
  'relief',
  'pride',
  'admiration',
  'desire',
  'caring'],
 'sadness': ['sadness', 'disappointment', 'embarrassment', 'grief', 'remorse'],
 'surprise': ['surprise', 'realization', 'confusion', 'curiosity']}

In [ ]:
emotion_file = open("/content/emotion_file.txt", "r")
emotion_list = emotion_file.read()
emotion_list = emotion_list.split("\n")
print(emotion_list)

['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']


In [ ]:
def idx2class(idx_list):
    arr = []
    for i in idx_list:
        arr.append(emotion_list[int(i)])
    return arr

In [ ]:
train_dataset['Emotions'] = train_dataset['Class List'].apply(idx2class)
valid_dataset['Emotions'] = valid_dataset['Class List'].apply(idx2class)
test_dataset['Emotions'] = test_dataset['Class List'].apply(idx2class)

In [ ]:
train_dataset["Emotions"].value_counts()


,count
Emotions,
[neutral],12823
[admiration],2710
[approval],1873
[gratitude],1857
[amusement],1652
...,...
"[confusion, gratitude, realization]",1
"[disappointment, disapproval, nervousness]",1
"[curiosity, disapproval, sadness]",1


In [ ]:
def EmotionMapping(emotion_list):
    map_list = []

    for i in emotion_list:
        if i in ekman_mapping['anger']:
            map_list.append('anger')
        if i in ekman_mapping['disgust']:
            map_list.append('disgust')
        if i in ekman_mapping['fear']:
            map_list.append('fear')
        if i in ekman_mapping['joy']:
            map_list.append('joy')
        if i in ekman_mapping['sadness']:
            map_list.append('sadness')
        if i in ekman_mapping['surprise']:
            map_list.append('surprise')
        if i == 'neutral':
            map_list.append('neutral')

    return map_list

In [ ]:
train_dataset['Mapped Emotions'] = train_dataset['Emotions'].apply(EmotionMapping)
valid_dataset['Mapped Emotions'] = valid_dataset['Emotions'].apply(EmotionMapping)
test_dataset['Mapped Emotions'] = test_dataset['Emotions'].apply(EmotionMapping)

In [ ]:
train_dataset["Mapped Emotions"].value_counts()

,count
Mapped Emotions,
[joy],12920
[neutral],12823
[anger],3878
[surprise],3553
"[joy, joy]",2129
...,...
"[fear, sadness, sadness]",1
"[joy, anger, disgust]",1
"[joy, fear, surprise]",1


In [ ]:
train_dataset.head()

,Text,Class,ID,Class List,Class Length,Emotions,Mapped Emotions
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1,[neutral],[neutral]
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1,[neutral],[neutral]
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1,[anger],[anger]
3,To make her feel threatened,14,ed7ypvh,[14],1,[fear],[fear]
4,Dirty Southern Wankers,3,ed0bdzj,[3],1,[annoyance],[anger]


In [ ]:
train_dataset['anger'] = np.zeros((len(train_dataset),1))
train_dataset['disgust'] = np.zeros((len(train_dataset),1))
train_dataset['fear'] = np.zeros((len(train_dataset),1))
train_dataset['joy'] = np.zeros((len(train_dataset),1))
train_dataset['sadness'] = np.zeros((len(train_dataset),1))
train_dataset['surprise'] = np.zeros((len(train_dataset),1))
train_dataset['neutral'] = np.zeros((len(train_dataset),1))

valid_dataset['anger'] = np.zeros((len(valid_dataset),1))
valid_dataset['disgust'] = np.zeros((len(valid_dataset),1))
valid_dataset['fear'] = np.zeros((len(valid_dataset),1))
valid_dataset['joy'] = np.zeros((len(valid_dataset),1))
valid_dataset['sadness'] = np.zeros((len(valid_dataset),1))
valid_dataset['surprise'] = np.zeros((len(valid_dataset),1))
valid_dataset['neutral'] = np.zeros((len(valid_dataset),1))

test_dataset['anger'] = np.zeros((len(test_dataset),1))
test_dataset['disgust'] = np.zeros((len(test_dataset),1))
test_dataset['fear'] = np.zeros((len(test_dataset),1))
test_dataset['joy'] = np.zeros((len(test_dataset),1))
test_dataset['sadness'] = np.zeros((len(test_dataset),1))
train_dataset['anger'] = np.zeros((len(train_dataset),1))
train_dataset['disgust'] = np.zeros((len(train_dataset),1))
train_dataset['fear'] = np.zeros((len(train_dataset),1))
train_dataset['joy'] = np.zeros((len(train_dataset),1))
train_dataset['sadness'] = np.zeros((len(train_dataset),1))
train_dataset['surprise'] = np.zeros((len(train_dataset),1))
train_dataset['neutral'] = np.zeros((len(train_dataset),1))

valid_dataset['anger'] = np.zeros((len(valid_dataset),1))
valid_dataset['disgust'] = np.zeros((len(valid_dataset),1))
valid_dataset['fear'] = np.zeros((len(valid_dataset),1))
valid_dataset['joy'] = np.zeros((len(valid_dataset),1))
valid_dataset['sadness'] = np.zeros((len(valid_dataset),1))
valid_dataset['surprise'] = np.zeros((len(valid_dataset),1))
valid_dataset['neutral'] = np.zeros((len(valid_dataset),1))

test_dataset['anger'] = np.zeros((len(test_dataset),1))
test_dataset['disgust'] = np.zeros((len(test_dataset),1))
test_dataset['fear'] = np.zeros((len(test_dataset),1))
test_dataset['joy'] = np.zeros((len(test_dataset),1))
test_dataset['sadness'] = np.zeros((len(test_dataset),1))
test_dataset['surprise'] = np.zeros((len(test_dataset),1))
test_dataset['neutral'] = np.zeros((len(test_dataset),1))

In [ ]:
for i in ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise','neutral']:
    train_dataset[i] = train_dataset['Mapped Emotions'].apply(lambda x: 1 if i in x else 0)
    valid_dataset[i] = valid_dataset['Mapped Emotions'].apply(lambda x: 1 if i in x else 0)
    test_dataset[i] = test_dataset['Mapped Emotions'].apply(lambda x: 1 if i in x else 0)

In [ ]:
train_dataset.head()

,Text,Class,ID,Class List,Class Length,Emotions,Mapped Emotions,anger,disgust,fear,joy,sadness,surprise,neutral
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1,[neutral],[neutral],0,0,0,0,0,0,1
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1,[neutral],[neutral],0,0,0,0,0,0,1
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1,[anger],[anger],1,0,0,0,0,0,0
3,To make her feel threatened,14,ed7ypvh,[14],1,[fear],[fear],0,0,1,0,0,0,0
4,Dirty Southern Wankers,3,ed0bdzj,[3],1,[annoyance],[anger],1,0,0,0,0,0,0


In [ ]:
train_dataset.drop(["Class", "ID", "Class List", "Class Length", "Emotions", "Mapped Emotions"], axis=1, inplace=True)
valid_dataset.drop(["Class", "ID", "Class List", "Class Length", "Emotions", "Mapped Emotions"], axis=1, inplace=True)
test_dataset.drop(["Class", "ID", "Class List", "Class Length", "Emotions", "Mapped Emotions"], axis=1, inplace=True)

In [ ]:
train_dataset.head()

,Text,anger,disgust,fear,joy,sadness,surprise,neutral
0,My favourite food is anything I didn't have to...,0,0,0,0,0,0,1
1,"Now if he does off himself, everyone will thin...",0,0,0,0,0,0,1
2,WHY THE FUCK IS BAYLESS ISOING,1,0,0,0,0,0,0
3,To make her feel threatened,0,0,1,0,0,0,0
4,Dirty Southern Wankers,1,0,0,0,0,0,0


In [ ]:
def preprocess(sentence):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    sentence = re.sub('[^A-z]', ' ', sentence)
    negative = ['not', 'neither', 'nor', 'but', 'however', 'although', 'nonetheless', 'despite', 'except', 'even though', 'yet']
    stop_words = [z for z in stop_words if z not in negative]
    preprocessed_tokens = [lemmatizer.lemmatize(contractions.fix(temp.lower())) for temp in sentence.split() if temp not in stop_words] #lemmatization
    return ' '.join([x for x in preprocessed_tokens]).strip()

def text_preprocessing_pipeline(text):
    '''Cleaning and parsing the text.'''
    text = preprocess(text)
    return text

In [ ]:
train_dataset['Text'] = train_dataset['Text'].apply(text_preprocessing_pipeline)
valid_dataset["Text"] = valid_dataset["Text"].apply(text_preprocessing_pipeline)
test_dataset["Text"] = test_dataset["Text"].apply(text_preprocessing_pipeline)

In [ ]:
pd.Series(train_dataset["Text"]).str.split().str.len().describe(percentiles=[0.05, 0.97])


,Text
count,43410.000000
mean,7.761253
std,3.956936
min,0.000000
5%,2.000000
50%,7.000000
97%,16.000000
max,33.000000


In [ ]:
train_dataset.head()


,Text,anger,disgust,fear,joy,sadness,surprise,neutral
0,my favourite food anything i cook,0,0,0,0,0,0,1
1,now everyone think he laugh screwing people in...,0,0,0,0,0,0,1
2,why the fuck is bayless isoing,1,0,0,0,0,0,0
3,to make feel threatened,0,0,1,0,0,0,0
4,dirty southern wanker,1,0,0,0,0,0,0


In [ ]:
train_labels = train_dataset.drop("Text", axis=1)
valid_labels = valid_dataset.drop("Text", axis=1)
test_labels = test_dataset.drop("Text", axis=1)

In [ ]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base", return_tensors="tf")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
def roberta_encode(data, maximum_length, tokenizer) :


    input_ids = []
    attention_masks = []
    token_type_ids = []
    for i in range(len(data['Text'])):

        encoded = tokenizer.encode_plus(data["Text"].iloc[i],
                                        add_special_tokens=True,
                                        max_length=maximum_length,
                                        pad_to_max_length=True,
                                        return_attention_mask=True,
                                        truncation=True,
                                        return_token_type_ids=True)

        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
        token_type_ids.append(encoded["token_type_ids"])


    return np.array(input_ids),np.array(attention_masks),np.array(token_type_ids)

In [ ]:
max_len = max([len(x.split()) for x in train_dataset['Text']])
train_input_ids,train_attention_masks, train_token_type_ids = roberta_encode(train_dataset, max_len, tokenizer)
valid_input_ids,valid_attention_masks, valid_token_type_ids = roberta_encode(valid_dataset, max_len, tokenizer)
test_input_ids,test_attention_masks, test_token_type_ids = roberta_encode(test_dataset, max_len, tokenizer)

In [ ]:
def create_model(bert_model, max_len, num_classes=7):
    input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')
    token_type_ids = tf.keras.Input(shape=(max_len,),dtype='int32')

    features = bert_model([input_ids,attention_masks, token_type_ids])
    fc_1 = tf.keras.layers.Dense(256, activation="relu")(features[1])
    dropout_1 = tf.keras.layers.Dropout(0.7)(fc_1)
    output = tf.keras.layers.Dense(num_classes, activation='sigmoid')(dropout_1)

    model = tf.keras.models.Model(inputs=[input_ids,attention_masks,token_type_ids], outputs=output)

    model.compile(tf.keras.optimizers.Adam(learning_rate=5e-5),
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.Precision(name='precision')])
    return model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)

In [ ]:
from transformers import TFRobertaModel
roberta_model = TFRobertaModel.from_pretrained('roberta-base')
roberta_model.trainable = False

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.dense.bias', 'roberta.embeddings.position_ids', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

In [ ]:
model = create_model(roberta_model, max_len)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 33)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 33)]         0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 33)]         0           []                               
                                                                                                  
 tf_roberta_model (TFRobertaMod  TFBaseModelOutputWi  124645632  ['input_1[0][0]',                
 el)                            thPoolingAndCrossAt               'input_2[0][0]',            

In [ ]:
history = model.fit([train_input_ids,train_attention_masks,train_token_type_ids], train_labels,
                    validation_data=([valid_input_ids,valid_attention_masks,valid_token_type_ids], valid_labels),
                    epochs=5,
                    batch_size=16,
                    callbacks=[es])

Epoch 1/5
1894/2714 [===================>..........] - ETA: 21:34 - loss: 0.4064 - accuracy: 0.3372 - precision: 0.3396

KeyboardInterrupt: 

In [ ]:
pred_labels = model.predict([test_input_ids,test_attention_masks, test_token_type_ids])
model.evaluate([test_input_ids,test_attention_masks, test_token_type_ids], test_labels)
def prob(x):
    if x>0.5:
        return 1
    return 0
pred_labels = [list(map(prob,i)) for i in pred_labels]
pred_labels[:5]

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from transformers import TFRobertaModel
es_2 = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
roberta_model = TFRobertaModel.from_pretrained('roberta-base')
roberta_model.trainable = True

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'roberta.embeddings.position_ids', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

In [ ]:
model_2 = create_model(roberta_model, max_len)
model_2.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 33)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 33)]         0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 33)]         0           []                               
                                                                                                  
 tf_roberta_model (TFRobertaMod  TFBaseModelOutputWi  124645632  ['input_1[0][0]',                
 el)                            thPoolingAndCrossAt               'input_2[0][0]',            

In [ ]:
history_2 = model_2.fit([train_input_ids,train_attention_masks,train_token_type_ids], train_labels,
                    validation_data=([valid_input_ids,valid_attention_masks,valid_token_type_ids], valid_labels),
                    epochs=5,
                    batch_size=16,
                    callbacks=[es_2])

Epoch 1/5
2714/2714 [==============================] - 468s 154ms/step - loss: 0.3070 - accuracy: 0.5711 - precision: 0.6575 - val_loss: 0.2526 - val_accuracy: 0.6325 - val_precision: 0.7585
Epoch 2/5
2714/2714 [==============================] - 395s 146ms/step - loss: 0.2643 - accuracy: 0.6392 - precision: 0.7182 - val_loss: 0.2463 - val_accuracy: 0.6419 - val_precision: 0.7254
Epoch 3/5
2714/2714 [==============================] - 399s 147ms/step - loss: 0.2453 - accuracy: 0.6682 - precision: 0.7423 - val_loss: 0.2395 - val_accuracy: 0.6463 - val_precision: 0.7232
Epoch 4/5
2714/2714 [==============================] - 398s 147ms/step - loss: 0.2288 - accuracy: 0.6944 - precision: 0.7647 - val_loss: 0.2522 - val_accuracy: 0.6461 - val_precision: 0.7089
Epoch 5/5
2714/2714 [==============================] - 399s 147ms/step - loss: 0.2107 - accuracy: 0.7276 - precision: 0.7930 - val_loss: 0.2636 - val_accuracy: 0.6439 - val_precision: 0.7042
Epoch 5: early stopping


In [ ]:
pred_labels_2 = model_2.predict([test_input_ids,test_attention_masks, test_token_type_ids])


170/170 [==============================] - 19s 80ms/step


In [ ]:
model_2.evaluate([test_input_ids,test_attention_masks, test_token_type_ids], test_labels)


170/170 [==============================] - 14s 83ms/step - loss: 0.2620 - accuracy: 0.6451 - precision: 0.7038


[0.26200515031814575, 0.6451078057289124, 0.7038140892982483]

In [ ]:
def prob(x):
    if x>0.5:
        return 1
    return 0

In [ ]:
pred_labels_2 = [list(map(prob,i)) for i in pred_labels_2]

In [ ]:
from sklearn.metrics import label_ranking_average_precision_score
lrap = label_ranking_average_precision_score(pred_labels_2, test_labels)
print("The Label Ranking Average Precision Score for the Test Set: ", lrap)

The Label Ranking Average Precision Score for the Test Set:  0.7445488606351189


In [ ]:
model_2.save("/content/drive/MyDrive/Emotion_classification_model_2.h5")